In [ ]:
import cv2
import json
import os
import numpy as np
import glob
import re
import time
from datetime import datetime
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from typing import Optional, List, Dict, Any, Tuple
import logging
from dataclasses import dataclass
import hashlib
import sys

# GUI Specific Imports
import customtkinter as ctk
from PIL import Image, ImageTk
from tkinter import messagebox
from pyzbar.pyzbar import decode
import pyttsx3
import threading
from tkinter import Toplevel
from queue import Queue, Empty

@dataclass
class Config:
    FACE_DB_PATH: str = "student_faces"
    LOG_FILE: str = "verification_log.json"
    STUDENT_DATA_FILE: str = "student_database.json"
    SUPPORTED_EXTENSIONS: List[str] = None
    
    # Camera settings
    CAMERA_WIDTH: int = 640
    CAMERA_HEIGHT: int = 480
    CAMERA_FPS: int = 25
    
    # Template Matching parameters
    TM_SCALE_FACTORS: List[float] = None
    TM_THRESHOLD: float = 0.6
    TM_METHOD: int = cv2.TM_CCOEFF_NORMED
    
    # PCA parameters (reduced for real-time)
    PCA_COMPONENTS: int = 30
    PCA_VARIANCE_RATIO: float = 0.85
    
    # Verification thresholds
    MATCH_THRESHOLD: float = 0.83
    REQUIRED_CONSECUTIVE: int = 3
    
    # Quality thresholds
    MIN_FACE_SIZE: int = 80
    PREFERRED_FACE_SIZE: int = 120

    VERIFICATION_TIMEOUT: int = 30  # seconds
    QR_SCAN_TIMEOUT: int = 15      # seconds
    
    # Real-time optimization
    FACE_SIZE: Tuple[int, int] = (64, 64)
    PROCESS_EVERY_N_FRAMES: int = 3  # Process every 3rd frame for speed
    MAX_TEMPLATES: int = 5  # Limit templates for speed
    
    def __post_init__(self):
        if self.SUPPORTED_EXTENSIONS is None:
            self.SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp']
        if self.TM_SCALE_FACTORS is None:
            self.TM_SCALE_FACTORS = [0.9, 1.0, 1.1]


class ResultLogger:
    def __init__(self, results_directory: str = "results"):
        self.results_dir = Path(results_directory)
        self.results_dir.mkdir(exist_ok=True)
        
        # Single file to store all results
        self.results_file = self.results_dir / "all_results.json"
        
    def save_result(self, 
                   member_name: str,
                   student_id: str, 
                   score: float,
                   runtime: float,
                   timestamp: Optional[str] = None,
                   algorithm_details: Dict[str, Any] = None) -> bool:
        """
        Save verification result by appending to accumulated results.
        """
        try:
            if timestamp is None:
                timestamp = datetime.now().isoformat()
            
            # Create result data with additional metadata
            result_data = {
                "member": member_name,
                "student_id": student_id,
                "score": round(score, 4),
                "runtime": round(runtime, 4),
                "timestamp": timestamp,
                "test_run": self._get_next_run_number(member_name),
                "algorithm_details": algorithm_details or {}
            }
            
            # Load existing results or create new list
            results = self._load_all_results()
            results.append(result_data)
            
            # Save back to file
            with open(self.results_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            
            print(f"✓ Result saved for {member_name}: Run #{result_data['test_run']}, Score={score:.3f}, Runtime={runtime:.3f}s")
            return True
            
        except Exception as e:
            print(f"✗ Failed to save result for {member_name}: {e}")
            return False
    
    def _load_all_results(self) -> List[Dict]:
        """Load all existing results"""
        if self.results_file.exists():
            try:
                with open(self.results_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except (json.JSONDecodeError, FileNotFoundError):
                return []
        return []
    
    def _get_next_run_number(self, member_name: str) -> int:
        """Get the next run number for a specific member"""
        results = self._load_all_results()
        member_results = [r for r in results if r.get('member') == member_name]
        return len(member_results) + 1


class RealTimeFaceTrainer:
    def __init__(self, config: Config):
        self.config = config
        self.pca = PCA(n_components=config.PCA_COMPONENTS)
        self.scaler = StandardScaler()
        self.face_cache = {}  # Cache for loaded face data
        self.template_cache = {}  # Cache for template images
        
    def load_face_images_for_student(self, student_id: str) -> Tuple[np.ndarray, List[np.ndarray]]:
        """Load and preprocess face images for a student in real-time"""
        if student_id in self.face_cache:
            return self.face_cache[student_id]
        
        face_dir = Path(self.config.FACE_DB_PATH) / student_id
        if not face_dir.exists():
            return np.array([]), []
        
        face_images = []
        template_images = []
        
        # Get all face image files
        image_files = []
        for ext in self.config.SUPPORTED_EXTENSIONS:
            image_files.extend(glob.glob(str(face_dir / f"*{ext}")))
        
        # Limit to prevent lag
        image_files = image_files[:20]  # Max 20 images
        
        for i, image_path in enumerate(image_files):
            try:
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                
                # Preprocess face
                processed = self.preprocess_face(img)
                face_images.append(processed.flatten())
                
                # Keep first few as templates
                if i < self.config.MAX_TEMPLATES:
                    template_images.append(img)
                    
            except Exception as e:
                logging.warning(f"Failed to load image {image_path}: {e}")
                continue
        
        face_array = np.array(face_images) if face_images else np.array([])
        
        # Cache the results
        self.face_cache[student_id] = (face_array, template_images)
        
        return face_array, template_images
    
    def preprocess_face(self, face_img: np.ndarray) -> np.ndarray:
        """Fast face preprocessing optimized for real-time"""
        if len(face_img.shape) == 3:
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = face_img.copy()

        # Resize to standard size
        resized = cv2.resize(gray, self.config.FACE_SIZE, interpolation=cv2.INTER_LINEAR)
    
        # Quick brightness analysis
        mean_brightness = np.mean(resized)
    
        # Simple brightness adjustment
        if mean_brightness > 180:
            resized = cv2.convertScaleAbs(resized, alpha=0.8, beta=-20)
        elif mean_brightness < 70:
            resized = cv2.convertScaleAbs(resized, alpha=1.2, beta=20)
    
        # Fast contrast enhancement
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
        enhanced = clahe.apply(resized)
    
        return enhanced
    
    def train_pca_for_student(self, student_id: str) -> Optional[Dict]:
        """Train PCA model for a specific student in real-time"""
        face_images, template_images = self.load_face_images_for_student(student_id)
        
        if len(face_images) == 0:
            return None
        
        try:
            # Adjust PCA components based on available data
            n_samples = len(face_images)
            n_features = face_images.shape[1]
            optimal_components = min(self.config.PCA_COMPONENTS, n_samples - 1, n_features)
            
            if optimal_components < 1:
                return None
            
            # Create PCA with optimal components
            pca = PCA(n_components=optimal_components)
            scaler = StandardScaler()
            
            # Fit and transform
            scaled_features = scaler.fit_transform(face_images)
            pca_features = pca.fit_transform(scaled_features)
            
            return {
                'pca': pca,
                'scaler': scaler,
                'face_features': pca_features,
                'face_labels': np.zeros(len(face_images), dtype=int),  # Single person
                'template_images': template_images,
                'student_id': student_id,
                'n_components': optimal_components,
                'training_samples': n_samples
            }
            
        except Exception as e:
            logging.error(f"Error training PCA for {student_id}: {e}")
            return None


class StudentSystem:
    def __init__(self, config: Config = None):
        self.config = config or Config()
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        self._setup_directories()
        self.load_student_data()
        
        self.trainer = RealTimeFaceTrainer(self.config)
        self.active_models = {}  # Cache for active student models
        self.frame_count = 0

    def _setup_directories(self):
        Path(self.config.FACE_DB_PATH).mkdir(exist_ok=True)

    def load_student_data(self):
        try:
            if Path(self.config.STUDENT_DATA_FILE).exists():
                with open(self.config.STUDENT_DATA_FILE, 'r', encoding='utf-8') as f:
                    self.student_data = json.load(f)
                self.logger.info(f"Loaded data for {len(self.student_data)} students")
            else:
                self.student_data = {}
                self.logger.info("No existing student data found")
        except Exception as e:
            self.logger.error(f"Error loading student data: {e}")
            self.student_data = {}

    def get_student_info(self, student_id: str) -> Optional[Dict[str, Any]]:
        return self.student_data.get(student_id)

    def validate_qr_data(self, qr_data: str) -> Tuple[Optional[str], Optional[str], Optional[str]]:
        student_id = qr_data.strip()
        student_info = self.get_student_info(student_id)
        if student_info:
            name = student_info.get("name", "N/A")
            program = student_info.get("program", "N/A")
            return student_id, name, program
        return None, None, None

    def prepare_student_model(self, student_id: str) -> bool:
        """Prepare model for student verification"""
        if student_id in self.active_models:
            return True
        
        model_data = self.trainer.train_pca_for_student(student_id)
        if model_data is None:
            self.logger.warning(f"Failed to prepare model for {student_id}")
            return False
        
        self.active_models[student_id] = model_data
        self.logger.info(f"Model prepared for {student_id}: {model_data['training_samples']} samples")
        return True

    def template_match_detection(self, frame: np.ndarray, student_id: str) -> List[Dict]:
        """Fast template matching for face detection"""
        if student_id not in self.active_models:
            return []
        
        model_data = self.active_models[student_id]
        template_images = model_data.get('template_images', [])
        
        if not template_images:
            return []
        
        detected_faces = []
        frame_height, frame_width = frame.shape[:2]
        
        # Use only first template for speed
        template = template_images[0]
        
        # Single scale template matching for speed
        if template.shape[0] < frame.shape[0] and template.shape[1] < frame.shape[1]:
            result = cv2.matchTemplate(frame, template, self.config.TM_METHOD)
            _, max_val, _, max_loc = cv2.minMaxLoc(result)
            
            if max_val > self.config.TM_THRESHOLD:
                x, y = max_loc
                w, h = template.shape[1], template.shape[0]
                
                # Quick corner check
                center_x, center_y = x + w//2, y + h//2
                corner_threshold = 0.15
                corner_w = int(frame_width * corner_threshold)
                corner_h = int(frame_height * corner_threshold)
                
                # Skip corner detections
                if not (center_x < corner_w or center_y < corner_h or 
                       center_x > (frame_width - corner_w) or center_y > (frame_height - corner_h)):
                    detected_faces.append({
                        'x': x, 'y': y, 'width': w, 'height': h,
                        'confidence': max_val, 'student_id': student_id
                    })
        
        return detected_faces

    def extract_face_features(self, face_img: np.ndarray, model_data: Dict) -> Optional[np.ndarray]:
        """Extract PCA features from face image"""
        try:
            processed = self.trainer.preprocess_face(face_img)
            flattened = processed.flatten().reshape(1, -1)
            scaled = model_data['scaler'].transform(flattened)
            features = model_data['pca'].transform(scaled)
            return features[0]
        except Exception as e:
            self.logger.error(f"Feature extraction error: {e}")
            return None

    def recognize_face(self, face_img: np.ndarray, student_id: str, threshold: float = 0.7) -> Tuple[str, float]:
        """Recognize face using trained model"""
        if student_id not in self.active_models:
            return "unknown", 0.0
        
        model_data = self.active_models[student_id]
        features = self.extract_face_features(face_img, model_data)
        
        if features is None:
            return "unknown", 0.0
        
        try:
            similarities = cosine_similarity([features], model_data['face_features'])[0]
            max_similarity = np.max(similarities)
            
            if max_similarity >= threshold:
                return student_id, max_similarity
            else:
                return "unknown", max_similarity
                
        except Exception as e:
            self.logger.error(f"Recognition error: {e}")
            return "unknown", 0.0

    def log_verification(self, student_id: str, name: str, program: str, success: bool, 
                        timestamp: str, score: float = 0.0, verification_time: float = 0.0):
        """Log verification attempt"""
        log_entry = {
            "student_id": student_id, "name": name, "program": program, "success": success,
            "timestamp": timestamp, "score": score, "verification_time": verification_time,
            "session_id": hashlib.md5(f"{timestamp}{student_id}".encode()).hexdigest()[:8]
        }
        
        try:
            logs = []
            if Path(self.config.LOG_FILE).exists():
                with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                    logs = json.load(f)
            logs.append(log_entry)
            
            # Keep only last 1000 logs
            if len(logs) > 1000:
                logs = logs[-1000:]
                
            with open(self.config.LOG_FILE, 'w', encoding='utf-8') as f:
                json.dump(logs, f, indent=2, ensure_ascii=False)
                
            status = "SUCCESS" if success else "FAILED"
            self.logger.info(f"Verification logged: {name} - {status} (Score: {score:.3f})")
            
        except Exception as e:
            self.logger.error(f"Error logging verification: {e}")

    def view_logs(self) -> str:
        """Return formatted logs for display"""
        try:
            if not Path(self.config.LOG_FILE).exists():
                return "No verification logs found."
            
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            
            if not logs:
                return "No verification logs found."
            
            # Show last 20 logs
            recent_logs = logs[-20:]
            output = "RECENT VERIFICATION LOGS\n" + "="*50 + "\n\n"
            
            for log in reversed(recent_logs):
                status = "✓ SUCCESS" if log['success'] else "✗ FAILED"
                timestamp = datetime.fromisoformat(log['timestamp']).strftime("%Y-%m-%d %H:%M:%S")
                score = log.get('score', 0) * 100
                
                output += f"{timestamp} | {status} | {log['name']} ({log['student_id']}) | Score: {score:.1f}%\n"
            
            return output
            
        except Exception as e:
            return f"Error loading logs: {e}"

    def export_verification_report(self) -> str:
        """Export verification report"""
        try:
            if not Path(self.config.LOG_FILE).exists():
                return "No verification logs found to export."
            
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            
            if not logs:
                return "No verification data to export."
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            report_file = f"verification_report_{timestamp}.json"
            
            total = len(logs)
            successful = sum(1 for log in logs if log['success'])
            success_rate = (successful / total * 100) if total > 0 else 0
            
            report = {
                'generated_at': datetime.now().isoformat(),
                'summary': {
                    'total_attempts': total,
                    'successful_verifications': successful,
                    'success_rate_percentage': round(success_rate, 2),
                },
                'recent_logs': logs[-50:]
            }
            
            with open(report_file, 'w', encoding='utf-8') as f:
                json.dump(report, f, indent=2, ensure_ascii=False)
                
            return f"Report successfully exported to {report_file}"
            
        except Exception as e:
            return f"Failed to generate report: {e}"


class RobustTTS:
    def __init__(self):
        self.speech_queue = Queue()
        self.worker_thread = threading.Thread(target=self._worker_loop, daemon=True)
        self.worker_thread.start()

    def _worker_loop(self):
        while True:
            try:
                text = self.speech_queue.get()
                if text is None:
                    break
                
                engine = pyttsx3.init(driverName='sapi5')
                voices = engine.getProperty('voices')
                if voices:
                    for voice in voices:
                        if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                            engine.setProperty('voice', voice.id)
                            break
                engine.setProperty('rate', 155)
                engine.setProperty('volume', 0.9)
                
                engine.say(text)
                engine.runAndWait()
                del engine
                
            except Exception as e:
                logging.error(f"TTS error: {e}")

    def speak(self, text: str):
        if text and text.strip():
            while not self.speech_queue.empty():
                try:
                    self.speech_queue.get_nowait()
                except Empty:
                    break
            self.speech_queue.put(text.strip())

    def cleanup(self):
        self.speech_queue.put(None)
        self.worker_thread.join(timeout=2.0)


class App(ctk.CTk):
    def __init__(self, student_system: StudentSystem):
        super().__init__()
        self.system = student_system
        self.tts_system = RobustTTS()
        self.result_logger = ResultLogger()
        
        self.verification_start_time = None
        self.member_name = "Member2"

        self.title("Student Verification System")
        self.geometry("1100x720")
        ctk.set_appearance_mode("Dark")
        ctk.set_default_color_theme("blue")
        
        # App State
        self.cap = None
        self.verification_active = False
        self.current_student_id = None
        self.current_student_name = None
        self.current_student_program = None
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.last_known_face = None
        self.frame_count = 0
        self.verification_step = "idle"
        self.qr_detected = False

        self.qr_scan_start_time = None
        self.max_verification_time = 30  # seconds
        self.max_qr_scan_time = 15 
        
        self.setup_gui()
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def setup_gui(self):
        # Layout
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # Sidebar
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(5, weight=1)
        
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Main Menu", 
                                      font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        self.verify_button = ctk.CTkButton(self.sidebar_frame, text="Verify Student", 
                                          command=self.verify_student_frame_event)
        self.verify_button.grid(row=1, column=0, padx=20, pady=10)

        self.logs_button = ctk.CTkButton(self.sidebar_frame, text="View Logs", 
                                        command=self.view_logs_frame_event)
        self.logs_button.grid(row=2, column=0, padx=20, pady=10)

        self.export_button = ctk.CTkButton(self.sidebar_frame, text="Export Report", 
                                          command=self.export_report_event)
        self.export_button.grid(row=3, column=0, padx=20, pady=10)

        self.exit_button = ctk.CTkButton(self.sidebar_frame, text="Exit", 
                                        fg_color="transparent", border_width=2,
                                        command=self.on_closing)
        self.exit_button.grid(row=6, column=0, padx=20, pady=20, sticky="s")
        
        # Verification Frame
        self.verify_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.verify_frame.grid_columnconfigure(0, weight=1)
        
        self.camera_label = ctk.CTkLabel(self.verify_frame, text="")
        self.camera_label.grid(row=0, column=0, padx=10, pady=10)

        self.status_label = ctk.CTkLabel(self.verify_frame, 
                                        text="Welcome! Click 'Start Verification' to begin.", 
                                        font=ctk.CTkFont(size=16))
        self.status_label.grid(row=1, column=0, padx=10, pady=5)
        
        self.info_label = ctk.CTkLabel(self.verify_frame, text="", 
                                      font=ctk.CTkFont(size=14))
        self.info_label.grid(row=2, column=0, padx=10, pady=5)
        
        self.progress_bar = ctk.CTkProgressBar(self.verify_frame, width=300)
        self.progress_bar.set(0)
        self.progress_bar.grid(row=3, column=0, padx=10, pady=5)

        self.start_button = ctk.CTkButton(self.verify_frame, text="Start Verification", 
                                         command=self.start_verification_process)
        self.start_button.grid(row=4, column=0, padx=10, pady=10)

        # Logs Frame
        self.logs_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.logs_frame.grid_columnconfigure(0, weight=1)
        self.logs_frame.grid_rowconfigure(0, weight=1)
        self.log_textbox = ctk.CTkTextbox(self.logs_frame, width=700, 
                                         font=("Courier New", 12))
        self.log_textbox.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        
        # Initial setup
        self.select_frame_by_name("verify")

    def select_frame_by_name(self, name):
        self.verify_frame.grid_remove()
        self.logs_frame.grid_remove()
        
        if name == "verify":
            self.verify_frame.grid(row=0, column=1, sticky="nsew")
        elif name == "logs":
            self.logs_frame.grid(row=0, column=1, sticky="nsew")

    def verify_student_frame_event(self):
        self.select_frame_by_name("verify")

    def view_logs_frame_event(self):
        self.select_frame_by_name("logs")
        logs = self.system.view_logs()
        self.log_textbox.configure(state="normal")
        self.log_textbox.delete("1.0", "end")
        self.log_textbox.insert("1.0", logs)
        self.log_textbox.configure(state="disabled")

    def export_report_event(self):
        message = self.system.export_verification_report()
        messagebox.showinfo("Export Report", message)

    def start_verification_process(self):
        if self.verification_active:
            return
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Camera Error", "Cannot open camera.")
            return

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.system.config.CAMERA_WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.system.config.CAMERA_HEIGHT)
        
        self.reset_verification_state()
        self.verification_active = True
        self.verification_step = "qr_scan"
        self.start_button.configure(state="disabled", text="Verification in Progress...")
        self.verification_start_time = time.time()
        self.qr_scan_start_time = time.time()
        
        self.update_frame()

    def reset_verification_state(self):
        self.current_student_id = None
        self.current_student_name = None
        self.current_student_program = None
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.last_known_face = None
        self.frame_count = 0
        self.progress_bar.set(0)
        self.qr_detected = False

    def stop_verification_process(self):
        self.verification_active = False
        self.verification_step = "idle"
        
        if self.cap:
            self.cap.release()
            self.cap = None
            
        self.start_button.configure(state="normal", text="Start Verification")
        self.reset_verification_state()

    def update_frame(self):
        if not self.verification_active:
            return

        if self.check_timeouts():
            return 
    
        ret, frame = self.cap.read()
        if not ret:
            self.after(33, self.update_frame)  # ~30 FPS
            return
        
        frame = cv2.flip(frame, 1)
        self.frame_count += 1
        
        self.display_timeout_info()

        if self.verification_step == "qr_scan":
            self.handle_qr_scan(frame)
        elif self.verification_step == "face_verify":
            self.handle_face_verify(frame)
        
        # Display frame
        self.display_frame(frame)
        self.after(33, self.update_frame)

    def display_frame(self, frame):
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        ctk_img = ctk.CTkImage(light_image=img, dark_image=img, 
                              size=(self.system.config.CAMERA_WIDTH, 
                                   self.system.config.CAMERA_HEIGHT))
        self.camera_label.configure(image=ctk_img)
        self.camera_label.image = ctk_img

    def handle_qr_scan(self, frame):
        self.status_label.configure(text="Step 1: Scan Student QR Code")
        self.info_label.configure(text="Position the QR code in the camera view.")

        decoded_objects = decode(frame)
        for obj in decoded_objects:
            qr_data = obj.data.decode("utf-8")
            student_id, name, program = self.system.validate_qr_data(qr_data)
            
            if all([student_id, name, program]) and not self.qr_detected:
                self.current_student_id = student_id
                self.current_student_name = name
                self.current_student_program = program
                self.qr_detected = True

                # Draw QR detection
                points = obj.polygon
                hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
                cv2.polylines(frame, [np.int32(hull)], True, (0, 255, 0), 3)
                
                self.status_label.configure(text="QR Code Detected! Preparing verification...")
                self.info_label.configure(text=f"Student: {name} - Loading model...")
                
                # Prepare face verification
                self.after(2000, self.prepare_face_verification)
                return

    def check_timeouts(self):
        """Check if any timeout conditions are met"""
        current_time = time.time()

        # Check QR scan timeout
        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            qr_elapsed = current_time - self.qr_scan_start_time
            if qr_elapsed > self.max_qr_scan_time:
                self.handle_timeout("qr_scan")
                return True

        # Check face verification timeout
        elif self.verification_step == "face_verify" and self.verification_start_time:
            verification_elapsed = current_time - self.verification_start_time
            if verification_elapsed > self.max_verification_time:
                self.handle_timeout("face_verify")
                return True

        return False

    def handle_timeout(self, timeout_type):
        """Handle different types of timeouts"""
        if timeout_type == "qr_scan":
            self.tts_system.speak("QR code scan timeout. Please try again.")
            messagebox.showwarning("Timeout", "QR code scanning timed out. Please try again.")
        elif timeout_type == "face_verify":
            self.tts_system.speak("Face verification timeout. Access denied.")
            messagebox.showwarning("Timeout", "Face verification timed out. Please try again.")
    
        # Log the failed verification
        if hasattr(self, 'current_student_id') and self.current_student_id:
            timestamp = datetime.now().isoformat()
            self.system.log_verification(
                self.current_student_id or "UNKNOWN",
                self.current_student_name or "Unknown User",
                self.current_student_program or "Unknown Program",
                False,  # success = False
                timestamp,
                score=self.best_score,
                verification_time=time.time() - self.verification_start_time if self.verification_start_time else 0
            )
    
        # Finalize as failed verification
        self.finalize_verification(False)

    def display_timeout_info(self):
        """Display remaining time information"""
        current_time = time.time()

        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            elapsed = current_time - self.qr_scan_start_time
            remaining = max(0, self.max_qr_scan_time - elapsed)
            self.info_label.configure(
                text=f"Position QR code in view. Time remaining: {remaining:.0f}s"
            )

        elif self.verification_step == "face_verify" and self.verification_start_time:
            elapsed = current_time - self.verification_start_time
            remaining = max(0, self.max_verification_time - elapsed)
            self.info_label.configure(
                text=f"Face verification in progress. Time remaining: {remaining:.0f}s"
            )

    def prepare_face_verification(self):
        if not self.current_student_id:
            return
            
        self.status_label.configure(text="Preparing face verification...", text_color="yellow")
        self.info_label.configure(text=f"Student: {self.current_student_name} ({self.current_student_program})")
        
        # Load model in real-time
        if not self.system.prepare_student_model(self.current_student_id):
            messagebox.showerror("Error", f"No face data found for {self.current_student_name}.")
            self.stop_verification_process()
            return
        
        self.verification_step = "face_verify"
        self.verification_start_time = time.time()

    def handle_face_verify(self, frame):
        threshold_percent = self.system.config.MATCH_THRESHOLD * 100
        self.status_label.configure(text=f"Step 2: Position Your Face Clearly (Threshold: {threshold_percent:.0f}%)")
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detected_faces = []
        
        # Process every N frames for performance
        if self.frame_count % self.system.config.PROCESS_EVERY_N_FRAMES == 0:
            detected_faces = self.system.template_match_detection(gray, self.current_student_id)
            
            if detected_faces:
                self.last_known_face = (
                    detected_faces[0]['x'], 
                    detected_faces[0]['y'], 
                    detected_faces[0]['width'], 
                    detected_faces[0]['height']
                )
        
        if self.last_known_face is not None:
            x, y, w, h = self.last_known_face
            
            # Extract face region
            face_img = gray[y:y+h, x:x+w]
            
            if face_img.size > 0:
                # Recognize face using real-time model
                recognized_name, confidence = self.system.recognize_face(
                    face_img, self.current_student_id, self.system.config.MATCH_THRESHOLD
                )
                
                # Update best score
                self.best_score = max(self.best_score, confidence)
                self.progress_bar.set(confidence)

                # Check if match is found
                if recognized_name == self.current_student_id and confidence > self.system.config.MATCH_THRESHOLD:
                    self.consecutive_matches += 1
                    status_text = f"MATCHING... {self.consecutive_matches}/{self.system.config.REQUIRED_CONSECUTIVE}"
                    status_color = "#32CD32"
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                else:
                    self.consecutive_matches = 0
                    status_text = "NO MATCH"
                    status_color = "red"
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
                # Display confidence score on frame
                score_text = f"{confidence*100:.1f}%"
                cv2.putText(frame, score_text, (x, max(y-10, 0)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                           (0, 255, 0) if confidence > self.system.config.MATCH_THRESHOLD else (0, 0, 255), 2)

                self.info_label.configure(
                    text=f"Score: {confidence*100:.1f}% | Best: {self.best_score*100:.1f}% | Status: {status_text}",
                    text_color=status_color
                )
        
                # Check for verification success
                if self.consecutive_matches >= self.system.config.REQUIRED_CONSECUTIVE:
                    self.finalize_verification(True)
            else:
                self.info_label.configure(text="Face region too small", text_color="orange")
        else:
            self.info_label.configure(text="No face detected. Please ensure good lighting.", text_color="orange")
            self.progress_bar.set(0)

    def finalize_verification(self, success):
        end_time = time.time()
        runtime = end_time - self.verification_start_time if self.verification_start_time else 0.0
        final_score = self.best_score 
        
        timestamp = datetime.now().isoformat()
        student_id = self.current_student_id
        student_name = self.current_student_name
        student_program = self.current_student_program
        
        # Log verification
        self.system.log_verification(
            student_id, student_name, student_program, success, timestamp,    
            score=final_score, verification_time=runtime
        )
        
        # Log result for member
        try:
            self.result_logger.save_result(
                member_name=self.member_name,
                student_id=student_id,
                score=final_score,
                runtime=runtime
            )
        except Exception as e:
            print(f"Failed to log result: {e}")
        
        # Stop verification process
        self.stop_verification_process()
        
        if success:
            self.status_label.configure(text="ACCESS GRANTED", text_color="green")
            self.info_label.configure(text=f"Welcome, {student_name}! Final Score: {final_score*100:.1f}%", text_color="green")
            self.show_student_details_screen(student_id, final_score)
        else:
            self.status_label.configure(text="ACCESS DENIED", text_color="red")
            self.info_label.configure(text=f"Face not recognized. Best score: {final_score*100:.1f}%", text_color="red")
            messagebox.showerror("Failure", "Face verification failed.")
            
        # Reset labels after delay
        self.after(5000, self.reset_labels)

    def show_student_details_screen(self, student_id: str, verification_score: float):
        """Display detailed screen with student information and voice announcement"""
        student_info = self.system.get_student_info(student_id)
        if not student_info:
            return
        
        # Create details window
        details_window = Toplevel(self)
        details_window.title("Verification Successful")
        details_window.geometry("600x500")
        details_window.configure(bg='#1a1a1a')
        details_window.resizable(False, False)
        
        # Make window modal and center it
        details_window.transient(self)
        details_window.grab_set()
        
        # Center the window
        details_window.update_idletasks()
        x = (details_window.winfo_screenwidth() // 2) - (300)
        y = (details_window.winfo_screenheight() // 2) - (250)
        details_window.geometry(f"600x500+{x}+{y}")
        
        # Create main frame
        main_frame = ctk.CTkFrame(details_window, fg_color="transparent")
        main_frame.pack(fill="both", expand=True, padx=20, pady=20)
        
        # Success header
        success_label = ctk.CTkLabel(
            main_frame, 
            text="VERIFICATION SUCCESSFUL",
            font=ctk.CTkFont(size=24, weight="bold"),
            text_color="#00ff00"
        )
        success_label.pack(pady=(0, 20))
        
        # Student photo frame
        photo_frame = ctk.CTkFrame(main_frame, width=180, height=200, fg_color="#2b2b2b")
        photo_frame.pack(pady=(0, 20))
        photo_frame.pack_propagate(False)
        
        # Try to load student photo
        photo_loaded = False
        try:
            card_photos_folder = Path("card_photos")
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                photo_path = card_photos_folder / f"{student_id}{ext}"
                if photo_path.exists():
                    img = cv2.imread(str(photo_path))
                    if img is not None:
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img_pil = Image.fromarray(img_rgb)
                        img_pil = img_pil.resize((170, 190), Image.Resampling.LANCZOS)
                        
                        photo_ctk = ctk.CTkImage(
                            light_image=img_pil, 
                            dark_image=img_pil, 
                            size=(170, 190)
                        )
                        
                        photo_label = ctk.CTkLabel(photo_frame, image=photo_ctk, text="")
                        photo_label.pack(expand=True, padx=5, pady=5)
                        photo_loaded = True
                        break
        except Exception as e:
            logging.warning(f"Could not load student photo: {e}")
        
        if not photo_loaded:
            photo_label = ctk.CTkLabel(
                photo_frame, 
                text="No Photo\nAvailable",
                font=ctk.CTkFont(size=14),
                text_color="#cccccc"
            )
            photo_label.pack(expand=True)
        
        # Student details
        details_frame = ctk.CTkFrame(main_frame, fg_color="#2b2b2b")
        details_frame.pack(fill="x", pady=(0, 20))
        
        name = student_info.get("name", "N/A")
        program = student_info.get("program", "N/A")
        batch = student_info.get("batch", "N/A")
        academic_performance = student_info.get("academic_performance", "N/A")
        cgpa = student_info.get("cgpa", "N/A")
        
        details_data = [
            ("Name:", name),
            ("Program:", program),
            ("Batch:", f"20{batch}" if batch.isdigit() else batch),
            ("Academic Performance:", academic_performance),
            ("CGPA:", cgpa),
            ("Verification Score:", f"{verification_score*100:.1f}%")
        ]
        
        for label, value in details_data:
            detail_frame = ctk.CTkFrame(details_frame, fg_color="transparent")
            detail_frame.pack(fill="x", padx=20, pady=5)
            
            label_widget = ctk.CTkLabel(detail_frame, text=label, 
                                       font=ctk.CTkFont(size=14, weight="bold"), anchor="w")
            label_widget.pack(side="left", fill="x", expand=True)
            
            value_widget = ctk.CTkLabel(detail_frame, text=str(value), 
                                       font=ctk.CTkFont(size=14), 
                                       text_color="#00bfff", anchor="e")
            value_widget.pack(side="right")
        
        # Close button
        button_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
        button_frame.pack(fill="x", pady=(10, 0))
        
        close_button = ctk.CTkButton(button_frame, text="Close", 
                                    command=details_window.destroy,
                                    fg_color="#2196F3", hover_color="#1976D2")
        close_button.pack(side="right")
        
        # Auto-close after 15 seconds
        def auto_close():
            try:
                if details_window.winfo_exists():
                    details_window.destroy()
            except:
                pass
        details_window.after(15000, auto_close)
        
        # Voice announcement
        self.announce_student_details(name, program, batch, academic_performance)

    def announce_student_details(self, name: str, program: str, batch: str, academic_performance: str):
        """Create and speak the announcement"""
        announcement_parts = [
            f"Verification successful.",
            f"Welcome, {name}.",
            f"Program: {program}.",
            f"Batch 20{batch}." if batch.isdigit() else f"Batch {batch}.",
            f"Academic performance: {academic_performance}.",
            "Access granted. Have a great day!"
        ]
        
        announcement_text = " ".join(announcement_parts)
        self.tts_system.speak(announcement_text)

    def reset_labels(self):
        self.status_label.configure(text="Welcome! Click 'Start Verification' to begin.", 
                                   text_color=("white", "black"))
        self.progress_bar.set(0)
        self.info_label.configure(text="")

    def on_closing(self):
        if self.verification_active:
            self.stop_verification_process()
    
        if self.cap and self.cap.isOpened():
            self.cap.release()
            self.cap = None
    
        try:
            if hasattr(self, 'tts_system'):
                self.tts_system.cleanup()
        except Exception as e:
            print(f"TTS cleanup failed: {e}")
    
        self.quit()
        self.destroy()
        sys.exit(0)


def main():
    """Main application entry point"""
    try:
        # Setup logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('system.log'),
                logging.StreamHandler()
            ]
        )
        
        # Initialize system
        config = Config()
        backend_system = StudentSystem(config)
        
        # Check if student database exists
        if not Path(config.STUDENT_DATA_FILE).exists():
            print(f"Warning: Student database {config.STUDENT_DATA_FILE} not found!")
            print("Please ensure the student database file exists.")
        
        # Check if face database exists
        if not Path(config.FACE_DB_PATH).exists():
            print(f"Warning: Face database directory {config.FACE_DB_PATH} not found!")
            print("Please ensure the face images directory exists.")
        
        # Start GUI application
        app = App(student_system=backend_system)
        print("Starting Student Verification System...")
        app.mainloop()
        
    except KeyboardInterrupt:
        print("\nSystem interrupted by user. Goodbye!")
    except Exception as e:
        logging.critical(f"Critical system error: {e}")
        print(f"Critical error: {e}")


if __name__ == "__main__":
    main()

2025-08-30 00:20:31,116 - INFO - Loaded data for 2 students


Starting Student Verification System...


2025-08-30 00:20:43,501 - INFO - Model prepared for S002: 20 samples
2025-08-30 00:20:49,817 - INFO - Verification logged: Kah Yung - SUCCESS (Score: 0.915)


✓ Result saved for Member2: Run #2, Score=0.915, Runtime=6.301s


SystemExit: 0

c:\Users\Yung\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
